**CNN_InceptionV3**

In [ ]:
import os
os.environ["KAGGLE_USERNAME"]="reviewc7"
os.environ["KAGGLE_KEY"]="483139e1b10d7096045ca1c60488c56b"

**CNN**

In [ ]:
import os
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

from keras.layers import Input, Lambda, Dense, Flatten,Dropout,Conv2D,MaxPooling2D, BatchNormalization
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!kaggle datasets download shashwatwork/knee-osteoarthritis-dataset-with-severity

 99% 202M/204M [00:11<00:00, 23.0MB/s]
100% 204M/204M [00:11<00:00, 19.1MB/s]


In [ ]:
file_name = '/content/knee-osteoarthritis-dataset-with-severity.zip'
with ZipFile(file_name, 'r') as zip_file:
  zip_file.extractall()

In [ ]:
first_class = []

base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

# Resizing images
classes = [0, 1, 2]
xdata = dict()
for klass in [0,1,2]:

    xdata[klass] = []
    class_dir = os.path.join(train_dir,str(klass))
    image_list =  os.listdir(class_dir)

    for sample in image_list:
        img = cv2.resize(cv2.imread(class_dir+'/'+sample),(100,100))
        xdata[klass].append(img)

In [ ]:
X = np.array(xdata[0] + xdata[1] + xdata[2])
print(X.shape)
Y = np.array([0 for i in range(len(xdata[0]))] + [1 for i in range(len(xdata[1]))] + [2 for i in range(len(xdata[2]))])
print(Y.shape)

(4848, 100, 100, 3)
(4848,)


In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3, random_state=0, shuffle=True)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes= np.unique(ytrain),
                                                  y= ytrain)
class_weights = dict(enumerate(class_weights))

In [ ]:
np.unique(ytrain)

array([0, 1, 2])

In [ ]:
def create_model():
  model=Sequential()
  model.add(Conv2D(64,(3,3),activation='relu',input_shape=(100,100,3)))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Conv2D(32,(3,3),activation='relu'))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(Dropout(0.5))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Conv2D(32,(3,3),activation='relu'))
  model.add(Dropout(0.5))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(3, activation='softmax'))

  return model

In [ ]:
cnn = create_model()
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(cnn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 49, 49, 64)        256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                        

In [ ]:
my_callbacks = [EarlyStopping(monitor='val_accuracy',patience=20, verbose=2, restore_best_weights=True)]

In [ ]:
cnn.fit(xtrain,ytrain,epochs=300,steps_per_epoch=117 ,batch_size=64,validation_data=(xtest,ytest),callbacks=my_callbacks)

Epoch 1/300
117/117 [==============================] - 21s 55ms/step - loss: 1.1728 - accuracy: 0.4347 - val_loss: 1.0612 - val_accuracy: 0.4749
Epoch 2/300
117/117 [==============================] - 5s 41ms/step - loss: 1.0645 - accuracy: 0.4730 - val_loss: 1.0532 - val_accuracy: 0.4770
Epoch 3/300
117/117 [==============================] - 4s 33ms/step - loss: 1.0454 - accuracy: 0.4847 - val_loss: 1.0542 - val_accuracy: 0.4770
Epoch 4/300
117/117 [==============================] - 4s 35ms/step - loss: 1.0345 - accuracy: 0.4982 - val_loss: 1.0425 - val_accuracy: 0.4873
Epoch 5/300
117/117 [==============================] - 4s 34ms/step - loss: 1.0316 - accuracy: 0.5010 - val_loss: 1.0362 - val_accuracy: 0.4873
Epoch 6/300
117/117 [==============================] - 4s 33ms/step - loss: 1.0146 - accuracy: 0.5166 - val_loss: 1.0304 - val_accuracy: 0.5107
Epoch 7/300
117/117 [==============================] - 4s 33ms/step - loss: 1.0149 - accuracy: 0.5179 - val_loss: 1.1525 - val_accuracy

In [ ]:
cnn.evaluate(xtest,ytest)

46/46 [==============================] - 1s 9ms/step - loss: 0.9379 - accuracy: 0.5890


[0.9378713965415955, 0.5890034437179565]

In [ ]:

import os
import numpy as np
from tensorflow.keras.preprocessing import image

# Make predictions on user input images
def predict_knee_image(user_image_path):
    user_image = image.load_img(user_image_path, target_size=(100, 100))

    # Check image format and convert if necessary
    if user_image.format != 'PNG':  # Adjust if you only support JPEG
        user_image = user_image.convert('RGB')

    user_image = image.img_to_array(user_image)
    user_image = np.expand_dims(user_image, axis=0)

    prediction = cnn.predict(user_image)
    predicted_class = np.argmax(prediction)

    if predicted_class == 0:
        print("Correct image: Knee image detected")
    else:
        print("Wrong image: Knee image not detected")

# Example usage
user_image_path = "/content/kidney.jpeg"  # Replace with the actual path
predict_knee_image(user_image_path)

1/1 [==============================] - 0s 17ms/step
Wrong image: Knee image not detected
